In [1]:
import os
import ee
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import math

In [2]:
# Create a project called "ee-sentinel-2temporal-nir"
# Google Account is jiawei.gao.polimi@gmail.com
ee.Authenticate(force = False)
ee.Initialize(project='ee-sentinel-2temporal-nir')

In [3]:
cwd = "c:\\Users\\m1865\\Desktop\\DISC"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [4]:
date_Start = "2023-01-01"
date_End = "2023-12-31"
name_Site = "IT-SR2"
roi_Distance = 100

In [5]:
# Read shapefile of ROI
gdf_ROI = gpd.read_file(cwd_Images_Processed + "\\" + name_Site + "\\" + str(roi_Distance) + "m.shp")
# Get its crs
EPSG_ROI = gdf_ROI.crs

In [6]:
# Create a GEE ROI in EPSG:4326
gdf_ROI_4326 = gdf_ROI.to_crs("EPSG:4326")
bounds = gdf_ROI_4326.bounds.values.reshape(-1)
roi = ee.Geometry.Rectangle([bounds[0],bounds[1],bounds[2],bounds[3]])

In [7]:
# Get the centroid of ROI
centroid = gdf_ROI.centroid
centroid_x = centroid.get_coordinates().iloc[0,0]
centroid_y = centroid.get_coordinates().iloc[0,1]
# Now we get all coordinates of pixels in our ROI
# Get all the coordinates of pixel inside ROI
arr_x = np.arange(centroid_x - roi_Distance / 2, centroid_x + roi_Distance / 2 + 10, 10)
arr_y = np.arange(centroid_y - roi_Distance / 2, centroid_y + roi_Distance / 2 + 10, 10)
arr_xy = np.empty(shape = (len(arr_x)*len(arr_y),2), dtype = int)
z = 0
for j in range(len(arr_y)):
    for i in range(len(arr_x)):
        arr_xy[z,0] = arr_x[i]
        arr_xy[z,1] = arr_y[j]
        z = z + 1
# Create a new geodataframe
df_Points = pd.DataFrame({
    'x': arr_xy[:,0],
    'y': arr_xy[:,1]
})
gdf_Points = gpd.GeoDataFrame(df_Points, geometry = gpd.points_from_xy(df_Points['x'], df_Points['y'], crs=EPSG_ROI))
gdf_Points['x'] = gdf_Points.geometry.x
gdf_Points['y'] = gdf_Points.geometry.y
gdf_Points

,x,y,geometry
0,603915.0,4842865.0,POINT (603915 4842865)
1,603925.0,4842865.0,POINT (603925 4842865)
2,603935.0,4842865.0,POINT (603935 4842865)
3,603945.0,4842865.0,POINT (603945 4842865)
4,603955.0,4842865.0,POINT (603955 4842865)
...,...,...,...
116,603975.0,4842965.0,POINT (603975 4842965)
117,603985.0,4842965.0,POINT (603985 4842965)
118,603995.0,4842965.0,POINT (603995 4842965)
119,604005.0,4842965.0,POINT (604005 4842965)


# Read Sentinel-2 L1C Image Collection

In [10]:
dataset_L1C = (
    ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
    .filterBounds(roi)
    .filterDate(date_Start,date_End)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
    .sort("system:time_start")
)
imageList_L1C = dataset_L1C.toList(dataset_L1C.size())
"There are " + str(dataset_L1C.size().getInfo()) + " L1C images available!"

'There are 161 L1C images available!'

In [23]:
idList_L1C = []
for i in range(dataset_L1C.size().getInfo()):
    temp_ID = dataset_L1C.getInfo()['features'][i]['id']
    temp_ID = temp_ID.split('/')[-1]
    idList_L1C.append(temp_ID)

In [24]:
dataset_L2A = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(roi)
    .filterDate(date_Start,date_End)
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))
    .sort("system:time_start")
)
imageList_L2A = dataset_L2A.toList(dataset_L2A.size())
dataset_L2A.size().getInfo()
"There are " + str(dataset_L2A.size().getInfo()) + " L2A images available!"

'There are 158 L2A images available!'

In [25]:
idList_L2A = []
for i in range(dataset_L2A.size().getInfo()):
    temp_ID = dataset_L2A.getInfo()['features'][i]['id']
    temp_ID = temp_ID.split('/')[-1]
    idList_L2A.append(temp_ID)

In [41]:
# Filter out the images that don't appear in both ImageCollections
idList_L1C_Redun = []
imageList_L1C_NoRedun = ee.List([])
for i in range(len(idList_L1C)):
    if idList_L1C[i] not in idList_L2A:
        idList_L1C_Redun.append(idList_L1C[i])
    else:
        temp_Image = ee.Image(imageList_L1C.get(i))
        imageList_L1C_NoRedun = imageList_L1C_NoRedun.add(temp_Image.getInfo())
redunList_L2A = []
imageList_L2A_NoRedun = ee.List([])
for i in range(len(idList_L2A)):
    if idList_L2A[i] not in idList_L1C:
        redunList_L2A.append(idList_L2A[i])
    else:
        temp_Image = ee.Image(imageList_L2A.get(i))
        imageList_L2A_NoRedun = imageList_L2A_NoRedun.add(temp_Image.getInfo())
redunList_L2A

['20230809T101609_20230809T102204_T32TNP',
 '20230811T101031_20230811T101028_T32TPP',
 '20230811T101031_20230811T101028_T32TNP',
 '20230928T101719_20230928T102123_T32TPP',
 '20231003T101841_20231003T102549_T32TPP']

In [44]:
imageList_L1C_NoRedun.size().getInfo()

153

In [45]:
imageList_L2A_NoRedun.size().getInfo()

153

In [ ]:
image_2 = ee.Image(imageList_L1C.get(0))
image_2.getInfo()['id']